In [ ]:
!uv sync
import numpy as np
from IPython.display import Image, HTML

from heights_clad.heights import roll_force_and_torque
from heights_clad.data_input import koeff_twip, koeff_ni_cold

#### Roll flattening - extension from the former notebook

For the former calculations, it was assumed that the radius of the working rolls is not changing. To get more accurate results, the elastic deformation of the roll, with the result of the temporary flattening of the roll in the roll gap. If the pressure distribution is known, the real flattening can be calculated numerically (Hertz'sche Pressung). A sketch of the flattening in the roll gap can be seen here:

![](Darstellung_reale_Abplattung.png)

There are certain approximations which can be taken, for example by Hitchcock, which replaced the real roll flattening curve through an elliptical pressure distribution which can be seen in the following figure on the left side and the depiction in the roll gap on the right side.

![](Hitchcock_Ersatzspannungsverteilung.png)

![](Abplattung_Hitchcock.png)

The equation by Hitchcock can be summed up by:
$$\dfrac{R'}{R_0} = 1 + C \cdot {\frac{F'_w }{b \, \Delta h}}$$
with $$C = {\dfrac{16 (1-\nu ^2)}{\pi E}}$$
with $R'$ = flattened roll radius, $R$ = unflattened roll radius, $F'_w$ = calculated roll force, $b$ = width of the work piece, $\nu$ = Poissonzahl des Walzenwerkstoffes, $E$ = E-Modul des Walzenwerkstoffes

Bohm, Flaxa and Hinkfoth extend this equation for ${R'}/{R_0} > 5.235$ relationships, therefore mainly for the cold rolling of thin strips and replaced the right side of the equation with:
$$ 2 \cdot \left(C \cdot {\frac{F'_w }{b \, \Delta h}} \right)^{2/3}$$

We use the values for rolls made of hardened steel: $E = 210 \, GPa$ and $\nu = 0.3$


In [ ]:
def flattening_hitchcock(
    roll_force_current: float, delta_height: float, width_work_piece: float
):
    youngs_modulus = 210e9
    poisson_ratio = 0.3
    c = 16 * (1 - poisson_ratio**2) / (np.pi * youngs_modulus)
    flattening_factor = c * roll_force_current / (width_work_piece * delta_height)
    if flattening_factor < 4.235:
        ratio = 1 + flattening_factor
    else:
        ratio = 2 * flattening_factor ** (2 / 3)
    return ratio


def flattened_radius(
    radius_starting: float,
    roll_force_current: float,
    delta_height: float,
    width_work_piece: float,
):
    ratio = flattening_hitchcock(roll_force_current, delta_height, width_work_piece)
    return radius_starting * ratio

Now we define the same starting parameters as last time and then calculate the rolling force and the rolling torque with the same equations as last time:

In [ ]:
height_layer_upper = 2e-3  # in [m]
height_layer_lower = 2e-3  # in [m]
temperature = 430 + 273.15  # in K
width = 50e-3  # in [m]
# Technical parameters
roll_velocity = 0.1047  # in [m/s]
friction_coefficient = 0.2  # [-]
roll_gap = 2e-3  # in [m]
radius = 0.1  # in [m]

In [ ]:
roll_force, roll_torque = roll_force_and_torque(
    radius,
    koeff_ni_cold,
    koeff_twip,
    height_layer_upper,
    height_layer_lower,
    temperature,
    width,
    roll_velocity,
    friction_coefficient,
    roll_gap,
)
roll_force, roll_torque

Now the flattened roll radius is calculated:

In [ ]:
new_roll_radius = flattened_radius(
    radius, roll_force, height_layer_upper + height_layer_lower - roll_gap, width
)
new_roll_radius

This is now done as part of a loop until the difference between the resulting radius and the previous radius is under a certain threshold.

In [ ]:
number_of_max_iterations = 10
i = 0
current_radius = radius
previous_radius = current_radius
while i < number_of_max_iterations:
    if (current_radius - previous_radius) > 1e-3 or i < 1:
        i += 1
        roll_force, roll_torque = roll_force_and_torque(
            current_radius,
            koeff_ni_cold,
            koeff_twip,
            height_layer_upper,
            height_layer_lower,
            temperature,
            width,
            roll_velocity,
            friction_coefficient,
            roll_gap,
        )
        previous_radius = current_radius
        current_radius = flattened_radius(
            radius,
            roll_force,
            height_layer_upper + height_layer_lower - roll_gap,
            width,
        )
        print(
            f"The previous radius was {previous_radius * 1e3:.2f} mm which resulted in a roll force of {roll_force / 1e3:.2f}. The newly calculated flattened radius is {current_radius * 1e3:.2f} mm."
        )
    else:
        break

print(f"Finished calculating after {i} iterations.")